In [5]:
from langchain.vectorstores import FAISS
import faiss

vectorstore = FAISS.from_documents(docs, embeddings)
faiss.write_index(vectorstore.index, "./.cache/faiss_index")



In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA, StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
import faiss

# Language Model
llm = ChatOpenAI()

# 캐시 디렉토리 설정
cache_dir = LocalFileStore("./.cache/")

# 텍스트 분할기 설정
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

# 문서 로드 및 분할
loader = UnstructuredFileLoader("./files/example.txt")
docs = loader.load_and_split(text_splitter=splitter)

# 임베딩 설정
embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

# FAISS 벡터 스토어 생성 및 저장
vectorstore = FAISS.from_documents(docs, embeddings)
faiss.write_index(vectorstore.index, "./.cache/faiss_index")

# Retriever 생성
retriever = vectorstore.as_retriever()

# ConversationBufferMemory 추가
memory = ConversationBufferMemory()

# 프롬프트 템플릿 정의
prompt_template = PromptTemplate(
    input_variables=["context"],
    template="Using the context below, answer the following question: {context}"
)

# LLMChain 생성
llm_chain = LLMChain(llm=llm, prompt=prompt_template)

# StuffDocumentsChain을 사용하여 문서 결합 체인 생성
combine_documents_chain = StuffDocumentsChain(llm_chain=llm_chain)

# RetrievalQA 체인 생성
chain = RetrievalQA(combine_documents_chain=combine_documents_chain, retriever=retriever, memory=memory)

# 체인 실행
# 질문 목록
questions = [
    "Is Aaronson guilty?",
    "What message did he write in the table?",
    "Who is Julia?"
]

# 질문에 대한 답변을 순차적으로 요청
for question in questions:
    response = chain.run(question)
    print(f"Question: {question}\nAnswer: {response}\n")

C:\Users\jihoo\AppData\Local\Temp\ipykernel_34492\1214550821.py:14: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI()
C:\Users\jihoo\AppData\Local\Temp\ipykernel_34492\1214550821.py:27: LangChainDeprecationWarning: The class `UnstructuredFileLoader` was deprecated in LangChain 0.2.8 and will be removed in 1.0. An updated version of the class exists in the langchain-unstructured package and should be used instead. To use it run `pip install -U langchain-unstructured` and import as `from langchain_unstructured import UnstructuredLoader`.
  loader = UnstructuredFileLoader("./files/example.txt")
C:\Users\jihoo\AppData\Local\Temp\ipykernel_34492\1214550821.py:31: LangChainDeprecationWarning: The class `Ope

Question: Is Aaronson guilty?
Answer: What is the significance of Winston's moment of weakness when he cries out for Julia in the passage?

Question: What message did he write in the table?
Answer: What task did Winston set out to do deliberately on the plank bed with the slate on his knees? 

Winston set out to re-educate himself by writing down his thoughts and beliefs, such as "FREEDOM IS SLAVERY" and "TWO AND TWO MAKE FIVE," in order to align his thinking with that of the Party.

Question: Who is Julia?
Answer: Who does Winston betray in a moment of weakness and desperation in this passage?

